In [49]:
import pandas as pd
import numpy as np
import pickle
from konlpy.tag import Mecab
import os
import collections
import re
from tqdm import tqdm
import nltk

mecab = Mecab() 

In [50]:
train_han = pd.read_csv('./my_data/han/train_data.csv', index_col=0)
val_han = pd.read_csv('./my_data/han/valid_data.csv', index_col=0)
test_han = pd.read_csv('./my_data/han/test_data.csv', index_col=0) 

In [51]:
train_han.cleaned_data = train_han.cleaned_data.apply(eval)
val_han.cleaned_data = val_han.cleaned_data.apply(eval)
test_han.cleaned_data = test_han.cleaned_data.apply(eval)

In [52]:
train_han['for tokenizer'] = train_han['for tokenizer'].apply(eval)
val_han['for tokenizer'] = val_han['for tokenizer'].apply(eval)
test_han['for tokenizer'] = test_han['for tokenizer'].apply(eval) 

In [30]:
train_lda = pd.read_csv('./my_data/lda/train_data.csv', index_col=0)
val_lda = pd.read_csv('./my_data/lda/valid_data.csv', index_col=0)
test_lda = pd.read_csv('./my_data/lda/test_data.csv', index_col=0) 

In [31]:
train_lda.cleaned_data = train_lda.cleaned_data.apply(eval)
val_lda.cleaned_data = val_lda.cleaned_data.apply(eval)
test_lda.cleaned_data = test_lda.cleaned_data.apply(eval)
train_lda['for tokenizer'] = train_lda['for tokenizer'].apply(eval)
val_lda['for tokenizer'] = val_lda['for tokenizer'].apply(eval)
test_lda['for tokenizer'] = test_lda['for tokenizer'].apply(eval) 

In [53]:
from collections import defaultdict
import re 
import sys
import os 
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten, Activation 
from keras.layers import Conv1D, MaxPooling1D, merge, Dropout, LSTM, GRU, Bidirectional, TimeDistributed
from keras.callbacks import EarlyStopping
from keras.models import Model 

from keras import optimizers
from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers as initializers, regularizers, constraints

max_sent_length= 100 
max_sentences = 30 
max_words_dic = 20000
embedding_dim = 100 
validation_split = 0.2 
adam = optimizers.Adam(lr=0.001) 
rmsprop = optimizers.RMSprop(lr=0.001) 

In [54]:
new_data = train_han[['for tokenizer','전체']].append(val_han[['for tokenizer','전체']])
new_data = new_data.append(test_han[['for tokenizer','전체']])
new_data  

,for tokenizer,전체
73688,"[유전, 자료, 기반, 혈통, 재구, 보정, 기법, 확립, 고밀, 칩, 기반, 유전...",유전형 자료 기반 혈통 재구성 및 보정기법 확립\n\n고밀도칩 기반 유전체 혈연행렬...
52162,"[회전자, 내경, 가공, 장치, 개발, 정확, 위치, 프레임, 부품, 고정, 개발,...",？ 회전자의 내경 가공 장치 개발\n\n - 정확한 위치에 프레임과 부품을 고정...
159853,"[연구, 공격, 취약점, 악용, 장치, 오작동, 탐지, 가상, 물리, 시스템, 내재...",본 연구에서는 알려진 또는 알려지지 않은 공격을 통해 취약점을 악용하는 IoT 장치...
113271,"[주인, 집, 경우, 집, 안, 반려, 견, 스트레스, 정서, 안정, 예방, 외부,...",- 주인이 집을 비울 경우 집 안에 홀로 남겨진 반려견의 스트레스와 정서적 불안정 ...
129806,"[미래, 응용, 서비스, 서비스, 품질, 형태, 요구, 자원, 측면, 다양, 예상,...",미래의 응용서비스는 서비스 품질/형태 및 요구자원의 측면에서 초다양화가 예상된다. ...
...,...,...
129097,"[미세, 구조, 제어, 기술, 비교, 분석, 압, 전, 소재, 입자, 형상, 입자,...",○ 미세구조 제어기술 비교분석(NKN계 압전소재) ○ BiTe계 입자형상 및 입자성...
42835,"[기술, 슈퍼, 컴, 활용, 인체, 척추, 정보, 데이터베이스, 정밀, 모델, 가상...","IT 기술/슈퍼컴을 활용하여 인체 척추 정보 데이터베이스, 3D 고정밀 모델과 가상..."
56045,"[생체, 합성, 소재, 적용, 최적, 설계, 기법, 활용, 골, 내, 안정, 개선,...","- 생체적합성이 우수한 신소재(Ti-6Al-7Nb)를 적용하고, 최적설계 기법을 활..."
42695,"[클라우드, 컴퓨팅, 환경, 향상, 단말, 기반, 기술, 사용, 추상, 네트워크, ...",클라우드 컴퓨팅 환경에서 Security 및 Privacy를 향상시키기 위해 단말에...


In [55]:
texts = list(new_data['for tokenizer']) 
tokenizer = Tokenizer(num_words=max_words_dic, lower=False, oov_token="<OOV>")
tokenizer.fit_on_texts(texts) 
word_dic = tokenizer.word_index

In [56]:
tr_documents = list(train_han.cleaned_data)
tr_labels = pd.get_dummies(train_han.label, prefix='label')
vl_documents = list(val_han.cleaned_data)
vl_labels = pd.get_dummies(val_han.label, prefix='label')
ts_documents = list(test_han.cleaned_data)
ts_labels = pd.get_dummies(test_han.label, prefix='label')

In [57]:
tr_data = np.zeros((len(tr_documents), max_sentences, max_sent_length), dtype='int32') 
vl_data = np.zeros((len(vl_documents), max_sentences, max_sent_length), dtype='int32')
ts_data = np.zeros((len(ts_documents), max_sentences, max_sent_length), dtype='int32')

In [60]:
# 데이터 구성 : [문서, 문장, 단어]
doc_tuple = [(tr_documents, tr_data),(vl_documents, vl_data),(ts_documents, ts_data)]


    
for i, sentences in tqdm(enumerate(ts_documents)): 
    for j, sentence in enumerate(sentences):
        if j < max_sentences:
            wordTokens = text_to_word_sequence(str(sentence))
            k = 0 
            for _, word in enumerate(wordTokens):
                try:
                    word = eval(word)
                    if k < max_sent_length :
                        if word not in tokenizer.word_index:
                            continue
                        if tokenizer.word_index[word] < max_sent_length:
                            ts_data[i,j,k] = tokenizer.word_index[word]
                            k+=1

                #except SyntaxError:
                except:
                    print(word)
                    if k < max_sent_length : 
                        if word not in tokenizer.word_index:
                            continue
                        if tokenizer.word_index[word] < max_sent_length:
                            ts_data[i,j,k] = tokenizer.word_index[word]
                            k+=1


3678it [00:07, 600.25it/s]

'내셔날
'


4318it [00:08, 502.55it/s]

'내셔날
'
'내셔날
'


6167it [00:12, 511.17it/s]


In [61]:
def dot_product(x, kernel):
    return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)


class AttentionWithContext(Layer):
    def __init__(self, W_regularizer=None, u_regularizer=None, b_regularizer=None,
                W_constraint=None, u_constraint=None, b_constraint=None, bias=True, **kwargs):
        
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')
        
        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)
        
        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)
        
        self.bias = bias 
        super(AttentionWithContext, self).__init__(**kwargs)
        
    def build(self, input_shape):
        assert len(input_shape) == 3
        self.W = self.add_weight(shape=(input_shape[-1], input_shape[-1],),
                                initializer=self.init, 
                                name='{}_W'.format(self.name),
                                regularizer=self.W_regularizer,
                                constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight(shape=(input_shape[-1],),
                                    initializer=self.init, 
                                    name='{}_b'.format(self.name),
                                    regularizer=self.b_regularizer,
                                    constraint=self.b_constraint)
        self.u = self.add_weight(shape=(input_shape[-1],),
                                initializer=self.init,
                                name='{}_u'.format(self.name),
                                regularizer=self.u_regularizer,
                                constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        return None
    
    def call(self, x, mask=None):
        uit = dot_product(x, self.W)
        if self.bias:
            uit += self.b
        
        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)
        
        a = K.exp(ait) 
        a /= K.cast(K.sum(a, axis=1, keepdims=True)+K.epsilon(), K.floatx())
        
        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)
    
    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]
    


In [62]:
data_X = [tr_data, vl_data, ts_data]
data_y = [tr_labels, vl_labels, ts_labels]

In [42]:
def run_model(data_x, data_y, max_sentences, max_sent_length):
    # 데이터
    x_train, x_valid, x_test = data_X[0], data_X[1], data_X[2]
    y_train, y_valid, y_test = data_y[0], data_y[1], data_y[2] 

    embedding_layer = Embedding(len(tokenizer.word_index)+1, embedding_dim,
                                input_length=max_sent_length, trainable=True)
    
    
    word_input = Input(shape=(max_sent_length,), dtype='int32')  
    word_sequences = embedding_layer(word_input) 
    word_lstm = Bidirectional(GRU(100, return_sequences=True))(word_sequences)
    word_dense = TimeDistributed(Dense(200))(word_lstm)
     word_att = AttentionWithContext()(word_dense)
    wordEncoder = Model(word_input, word_att)
    
    sent_input = Input(shape=(max_sentences, max_sent_length), dtype='int32')
    sent_encoder = TimeDistributed(wordEncoder)(sent_input)
    sent_lstm = Bidirectional(GRU(100, return_sequences=True))(sent_encoder)
    sent_dense = TimeDistributed(Dense(200))(sent_lstm)
    sent_att = AttentionWithContext()(sent_dense)
    latent_representation_of_doc = Dense(2)(sent_att)
    predictions = Dense(2, activation='softmax')(sent_att)
    model = Model(sent_input, predictions)
    #Encoder = Model(sent_input, latent_representation_of_doc)
    
    model.compile(loss ='categorical_crossentropy', 
                 optimizer = adam,
                 metrics = ['acc'])
    
    print(model.summary())
    
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
    history = model.fit(x_train, y_train, validation_data=(x_valid, y_valid),
                       epochs=2, batch_size=50, callbacks=[es])
    
    loss, acc = model.evaluate(x_train, y_train, verbose=2)
    print('Training Acc: {:.4f}'.format(acc)) 
    
    loss, acc = model.evaluate(x_test, y_test, verbose=2)
    print('Testing Acc: {:.4f}'.format(acc))
    
    #y_pred = np.argmax(model.predict(x_test), axis=1) +1
    #cm = confusion_matrix(y_test, y_pred, labels=None, sample_weight=None)
    
    return model, history
        

In [43]:
model, history = run_model(data_X, data_y, max_sentences, max_sent_length) 

Model: "functional_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 30, 100)]         0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 30, 200)           2468990   
_________________________________________________________________
bidirectional_3 (Bidirection (None, 30, 200)           181200    
_________________________________________________________________
time_distributed_5 (TimeDist (None, 30, 200)           40200     
_________________________________________________________________
attention_with_context_3 (At (None, 200)               40400     
_________________________________________________________________
dense_7 (Dense)              (None, 2)                 402       
Total params: 2,731,192
Trainable params: 2,731,192
Non-trainable params: 0
____________________________________________

In [44]:
ict_prob = model.predict(ts_data) 
han_df = pd.DataFrame(ts_labels)
han_df = pd.merge(han_df, new_data, left_index=True, right_index=True, how='left')
han_df['ICT_prob'] = ict_prob[:,1]
han_df = han_df.sort_values(by='ICT_prob', ascending=False)
y_pred= [1 if prob >= 0.5 else 0 for prob in han_df.ICT_prob]
han_df

,label_0,label_1,for tokenizer,전체,ICT_prob
1113,0,1,"[차년, 개발, 기반, 브로커, 개발, 구성, 요소, 간, 다양, 연동, 테스트, ...",1차년도에 개발된 Thin-Agent와 Connected Resource Manag...,0.955209
103341,0,1,"[당사, 푸시, 서비스, 개발, 확보, 다운스, 트림, 통신, 기술, 바탕, 업, ...",당사가 푸시 서비스를 개발하면서 확보한 다운스트림 통신 기술을 바탕으로 업스트림 통...,0.951943
145858,0,1,"[지능, 투어, 가이드, 시스템, 이하, 시스템, 투어, 가이드, 앱, 방문자, 실...",지능형 투어 가이드 시스템(이하 ITG(Intelligent Tour Guide)시...,0.943554
1168,0,1,"[정형, 정형, 형태, 데이터, 수집, 표준, 포맷, 변환, 고성능, 처리, 기능,...","ㅇ 정형, 비정형의 어떠한 형태의 데이터도 수집하여 표준화 된 포맷으로 변환하여 고...",0.942266
58304,0,1,"[기술, 개발, 최종, 목표, 선박, 승선, 여행, 여객, 선원, 육상, 가족, 연...","1. 기술개발 최종목표\n\n\n\n 선박을 승선하여 여행하는 여객이나, 선원,...",0.942209
...,...,...,...,...,...
14583,1,0,"[연구, 저온, 신호, 인식, 전달, 기능, 사이토, 키닌, 중, 사이토, 키닌, ...",본 연구는 저온 신호의 인식과 전달 기능을 가지고 있는 사이토키닌 twocompon...,0.000580
136750,1,0,"[신장, 질환, 핵, 수용체, 이용, 신장, 번역, 후, 변형, 기능, 신규, 표,...",신장질환에서 핵 수용체 FXR을 이용하여 <신장 FXR의 번역 후 변형 기능과 신규...,0.000576
25130,1,0,"[유전자, 발현, 조절, 기능, 발현, 조절, 기능, 분리, 인식, 이, 단계, 관...",유전자 발현 조절에 기능하는 STA1-mediated RNA regulatory n...,0.000523
162509,1,0,"[연구, 개, 유전자, 포함, 애기장대, 단백질, 유전자, 중, 개, 유전자, 대상...",본 연구에서는 42개 유전자를 포함하는 애기장대 non-tandem CCCH-typ...,0.000511


In [ ]:
han_df[['전체','label_1','ICT_prob']].to_csv('./data/ntis_classification_by_han_Data.csv')

In [26]:
han_df = pd.read_csv('./my_data/han/ntis_classification_by_han_Data.csv', index_col=0)
han_df

,전체,label_1,ICT_prob
135888,4차 산업 혁명시대의 사물 인터넷 인프라에 호환이 되며 최적의 적합성을 가진 스마트...,1,0.999179
161870,- 각종 도로 및 철도 차량 대상으로 5G 기반 통신 서비스를 제공하기 위한 Veh...,1,0.998625
33307,클라우드 컴퓨팅은 유비쿼터스 환경으로 급변하는 현재 지식서비스 분야에서 가장 유망한...,1,0.998605
57743,"？ 본 사업은 농업 IT 융합 기술인 도시형 인삼, 약용 식물 공장을 위한 센서네트...",1,0.998557
1026,제안 기술의 개요 및 개발 방향 \n\n\n\n- 퀄컴사의 오픈 플랫폼인 Allj...,1,0.998459
...,...,...,...
124737,"일산화질소 (nitric oxide, NO)는 생체 내에서 생리활성물질로서 다양한 ...",0,0.054581
182797,최종 목표: 구강암 발생 마우스 모델 이용 G 단백질 연결 수용체 54(G prot...,0,0.051975
182555,조기 재활치료와 뇌졸중 환자의 기능 회복이 보편적으로 양의 상관관계가 있다고 생각되...,0,0.051806
22062,연구내용\n 1. EST data base를 이용한 새로운 isoform의 xe...,0,0.048991


In [46]:
from sklearn.metrics import classification_report
print(classification_report(han_df.label_1, y_pred)) 

              precision    recall  f1-score   support

           0       0.87      0.92      0.90     29122
           1       0.63      0.51      0.57      7928

    accuracy                           0.83     37050
   macro avg       0.75      0.72      0.73     37050
weighted avg       0.82      0.83      0.82     37050



In [ ]:
from sklearn.metrics import auc, roc_curve
import matplotlib.pyplot as plt
fpr, tpr, thresholds = roc_curve(lda_df.label_1, y_pred)
roc_auc = auc(fpr, tpr)


plt.figure()
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--') 
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC_curve')
plt.legend(loc="lower right")
plt.show() 

In [ ]:
from sklearn.metrics import auc, roc_curve
import matplotlib.pyplot as plt
fpr, tpr, thresholds = roc_curve(latent_representation.label_1, y_pred)
roc_auc = auc(fpr, tpr)


plt.figure()
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC_curve')
plt.legend(loc="lower right")
plt.show() 

### 뉴스  데이터 테스트

In [91]:
news_df = pd.read_csv('./data/최종본_KBS_2017.csv')
news_df = news_df.drop_duplicates(['content'])
news = news_df[['content']] 

In [92]:
def preprocessing(text_list):
    preprocessed_text = []
    for text in text_list:
        text = re.sub(r'\W+', ' ', text)
        text = re.sub('o','', text)
        text = re.sub('○','', text)
        text = re.sub('ㅇ','', text)
        text = mecab.nouns(text)
        preprocessed_text.append(text)
    return preprocessed_text



In [93]:
news.content = news.content.apply(lambda text: nltk.sent_tokenize(text))
news.content = news.content.apply(preprocessing) 
news

/Users/kiki/miniconda3/lib/python3.8/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,content
0,"[[앵커, 트, 치열, 경쟁, 미래, 앞, 우리, 청년, 삶], [이, 건, 긍정,..."
1,"[[앵커, 트, 이번, 곳, 철강, 생산, 현장, 연결], [포항, 제철소, 김수영..."
2,"[[앵커, 트, 새해, 첫날, 해, 기원, 소망, 다짐, 가슴, 분], [서울, 광..."
3,"[[앵커, 트, 지금, 우리, 경제, 주력, 산업, 도약, 현장, 이번, 앞, 우리..."
4,"[[앵커, 트, 최문종, 오프닝, 공, 웅조, 이곳, 서울, 명동], [시간, 가족..."
...,...
7493,"[[년, 마지막], [이제, 올해, 시간], [새해, 첫날, 내일, 해돋이, 계획,..."
7494,"[[앵커, 트, 올해, 마지막, 날, 오늘, 광주, 아파트, 불, 나, 아이, 명,..."
7495,"[[올해, 마지막, 날], [밤, 사이, 눈, 비, 미세먼지, 농도, 대기], [수..."
7496,"[[앵커, 트, 올해, 마지막, 날, 광주, 아파트, 불, 나, 아이, 명, 사고]..."


In [94]:
data = np.zeros((len(news), max_sentences, max_sent_length), dtype='int32') 
documents = list(news.content)


for i, sentences in tqdm(enumerate(documents)):
    for j, sentence in enumerate(sentences): 
        if j < max_sentences:
            wordTokens = text_to_word_sequence(str(sentence))
            k = 0 
            for _, word in enumerate(wordTokens):
                try:
                    word = eval(word)
                    if k < max_sent_length :
                        if word not in tokenizer.word_index:
                            continue
                        if tokenizer.word_index[word] < max_sent_length:
                            data[i,j,k] = tokenizer.word_index[word]
                            k+=1
                    
                except SyntaxError:
                    print(word)
                    if k < max_sent_length :
                        if word not in tokenizer.word_index:
                            continue
                        if tokenizer.word_index[word] < max_sent_length:
                            data[i,j,k] = tokenizer.word_index[word]
                            k+=1

6229it [00:05, 1135.89it/s]


In [95]:
Ict_prob = model.predict(data)
news['ICT_prob'] = Ict_prob[:,1]
news

<ipython-input-95-956ef803a3bf>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news['ICT_prob'] = Ict_prob[:,1]


,content,ICT_prob
0,"[[앵커, 트, 치열, 경쟁, 미래, 앞, 우리, 청년, 삶], [이, 건, 긍정,...",0.136031
1,"[[앵커, 트, 이번, 곳, 철강, 생산, 현장, 연결], [포항, 제철소, 김수영...",0.087283
2,"[[앵커, 트, 새해, 첫날, 해, 기원, 소망, 다짐, 가슴, 분], [서울, 광...",0.010626
3,"[[앵커, 트, 지금, 우리, 경제, 주력, 산업, 도약, 현장, 이번, 앞, 우리...",0.169594
4,"[[앵커, 트, 최문종, 오프닝, 공, 웅조, 이곳, 서울, 명동], [시간, 가족...",0.007267
...,...,...
7493,"[[년, 마지막], [이제, 올해, 시간], [새해, 첫날, 내일, 해돋이, 계획,...",0.076800
7494,"[[앵커, 트, 올해, 마지막, 날, 오늘, 광주, 아파트, 불, 나, 아이, 명,...",0.359094
7495,"[[올해, 마지막, 날], [밤, 사이, 눈, 비, 미세먼지, 농도, 대기], [수...",0.054395
7496,"[[앵커, 트, 올해, 마지막, 날, 광주, 아파트, 불, 나, 아이, 명, 사고]...",0.280817


In [96]:
news['original_news'] = news_df.content
news

<ipython-input-96-fcbf8ba1d78d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news['original_news'] = news_df.content


,content,ICT_prob,original_news
0,"[[앵커, 트, 치열, 경쟁, 미래, 앞, 우리, 청년, 삶], [이, 건, 긍정,...",0.136031,<앵커 멘트> 치열한 경쟁과 불확실한 미래 앞에 서 있는 우리 청년들의 삶은 만만치...
1,"[[앵커, 트, 이번, 곳, 철강, 생산, 현장, 연결], [포항, 제철소, 김수영...",0.087283,<앵커 멘트> 이번엔 어느 곳보다 뜨거운 철강 생산 현장을 연결합니다. 포항제철소에...
2,"[[앵커, 트, 새해, 첫날, 해, 기원, 소망, 다짐, 가슴, 분], [서울, 광...",0.010626,"<앵커 멘트> 새해 첫날, 보다 나은 한 해를 기원하며 소망과 다짐을 가슴에 새긴 ..."
3,"[[앵커, 트, 지금, 우리, 경제, 주력, 산업, 도약, 현장, 이번, 앞, 우리...",0.169594,<앵커 멘트> 지금까지 우리 경제를 이끌어왔던 주력 산업의 재도약 현장들을 보셨는데...
4,"[[앵커, 트, 최문종, 오프닝, 공, 웅조, 이곳, 서울, 명동], [시간, 가족...",0.007267,"<앵커 멘트> <최문종오프닝+공웅조> 네, 이곳은 서울 명동입니다. 늦은 시간인데도..."
...,...,...,...
7493,"[[년, 마지막], [이제, 올해, 시간], [새해, 첫날, 내일, 해돋이, 계획,...",0.076800,2017년의 마지막 해도 저물었습니다. 이제는 올해가 5시간도 채 남지 않았는데요....
7494,"[[앵커, 트, 올해, 마지막, 날, 오늘, 광주, 아파트, 불, 나, 아이, 명,...",0.359094,<앵커 멘트> 올해 마지막 날인 오늘 광주의 한 아파트에 불이 나 어린 아이 세 명...
7495,"[[올해, 마지막, 날], [밤, 사이, 눈, 비, 미세먼지, 농도, 대기], [수...",0.054395,"어느덧 올해 마지막 날입니다. 밤사이 내린 눈,비로 미세먼지 농도가 옅어지긴 했지만..."
7496,"[[앵커, 트, 올해, 마지막, 날, 광주, 아파트, 불, 나, 아이, 명, 사고]...",0.280817,<앵커 멘트> 올해 마지막 날 광주의 한 아파트에 불이 나 어린 아이 세 명이 숨지...


In [97]:
news = news.sort_values(by='ICT_prob',ascending=False)

In [98]:
news

,content,ICT_prob,original_news
2551,"[[앵커, 트, 부모, 독립, 결혼, 여기, 고령, 영향, 인, 가구], [앞, 년...",0.845094,"<앵커 멘트> 부모로부터 독립해 살고, 결혼이 늦어지고 여기에 고령화의 영향까지 겹..."
731,"[[앵커, 트, 뷰티], [한국, 화장품, 한류, 영향, 아시아, 시장, 석권, 여...",0.821510,<앵커 멘트> 'K-뷰티'라고 들어보셨지요. 한국 화장품이 한류 영향으로 아시아 시...
971,"[[앵커, 트, 미국, 보호, 무역, 주, 강화, 중국, 사드, 보복, 조치, 등,...",0.782220,<앵커 멘트> 미국의 보호무역주의 강화와 중국의 사드 보복 조치 등으로 수출 환경이...
7092,"[[앵커, 트, 우리, 국민, 명, 중, 명, 암유, 병자, 것], [년, 기준, ...",0.779282,<앵커 멘트> 우리 국민 31명 중 1명은 '암유병자'인 것으로 나타났습니다. 20...
6781,"[[앵커, 트, 정부, 임대, 주택, 등록, 활성, 임대, 주택, 사업자, 등록, ...",0.773388,<앵커 멘트> 정부가 임대주택 등록을 활성화하기 위해 임대주택 사업자로 등록할 경우...
...,...,...,...
2064,"[[앵커, 트, 서울, 병원, 남자, 간호사, 휴대, 전화, 동료, 여성, 간호사,...",0.001003,<앵커 멘트> 서울의 한 병원에서 남자 간호사가 휴대전화로 동료 여성 간호사들의 신...
2069,"[[앵커, 트, 병원, 남자, 간호사, 휴대, 전화, 여자, 간호사, 신체, 적발]...",0.001003,<앵커 멘트> 한 병원에서 남자 간호사가 휴대전화로 여자 간호사들의 신체를 몰래 찍...
3804,"[[앵커, 트, 십, 대, 또래, 학생, 구타, 영상, 촬영, 유포, 일, 우리나라...",0.000983,"<앵커 멘트> 십 대들이 또래 학생을 구타하고 영상을 촬영해 유포하는 일, 우리나라..."
5139,"[[앵커, 트, 부산, 경찰청, 불법, 몰래카메라, 경고, 영상, 몰카, 유통, 사...",0.000958,<앵커 멘트> 부산경찰청이 불법 몰래카메라를 가장한 경고 영상을 만들어 몰카가 유통...


In [ ]:
news[['original_news','ICT_prob']].to_csv('./data/news_classification_by_han_Data.csv', header=True, encoding='utf-8-sig',index=False) 